# Decision Tree for classification

## 1. Motivation

First of all, **Decision Tree** is a tool for making predicitions. The motivation is to built a flowchart like a tree structure to show how decisions are made in steps. Here is an exmaple (by anshul508). This dataset provides ten observations, and the to-be-predicted (dependent) variable is whether to play golf or not. (Note that this is a classification problem).

> ![eg data](https://editor.analyticsvidhya.com/uploads/905753.png)

Decision trees are nothing but a bunch of if-else statements in layman terms. It checks if the condition is true and if it is then it goes to the next node attached to that decision.
> ![decision tree](https://editor.analyticsvidhya.com/uploads/542834.png)

In the above example, The **root node** is the upmost rectangal labelled by a **feature** called "weather". The root node then **split**s into three **branch**es (a.k.a **subtree**s).  We see that if the weather is cloudy then we must go to play, and it terminates the decision process. Node as such is called a **leaf node**. If, however the weather is sunny, we reach a new **decision node** labelled by a new feature called "humidity".

# 2 How to build (calibrate) a decision tree

Okay, the idea of a decision tree is not hard to understand, but how can we build a decision tree from scratch? Well, you need to have a dataset in the first place. 

Since we only focus on a classification decision tree, our goal is to distinguish target=-1 from target=1 using information in the sample. And if the sample is representative, we can use the same tree to make predictions about the population.

With this in mind, we need to develope an algorithm to find the "Best" tree. Or more specifically, **the "Best" sequence of splittings**.

## 2.1 What is "Best"?
To find the best sequence of splitting, we need to first define what is "good". **A "good" splitting should lead to purer and more accurate information**. (If a prediction is invariable 0.5 probability of positive and 0.5 probability of negative, it will add no value to our decision.) . 

As an example, consider splitting the origional data using variable "Humidity". After the spliting, 7 observations are in the "High" subset, and 3 of them are labeled "Yes". On the contrary, if we split the original data using "Weather", 3 observations are in the "Cloudy" subset, and all of them have a "Yes" label. Information provided by the "Weather=Cloudy" event is purer than the "Humidity=High" event, and therefore "Weather" maybe a better variable to split the data.

In light of this, we first introduce two measures of information "impurity", so that we can mathematically measure the quality of a splitting.

## 2.2 Measure impurity of a set 1 - Gini impurity

The decision tree construction starts with measuring the "**impurity**" of a single set of data. A split would divide the original sample into several subsamples. If the subsamples are more sorted and organized compared to the original sample, then this is a good split. To measure the sortedness, we take advantage of measures from information science that are used to measure inpurity of a set of data. 

We define $S$ as a set of data, and $j = 1,2,..,J$ as all possible labels of the variable (There are two labels: "Yes" and "No" in the example, so $J$=2). $p_j$ denotes the probability of the variable taking label $j$. And if we are dealing with a sample, the empirical probability $p_j$ can be defined as the the number of observations labelled with $j$ divided by the total number of obervations.

$$G(S) = \sum_{j=1}^J p_j(1-p_j)$$

Gini is perhaps the most intuitive measure of inpurity if you are familiar with probability theory. $p_j(1-p_j)$ is the variances of a Bernouli distribution which obtains the largest value when $p_j=0.5$.

## 2.3 Measure of impurity of a set 2 - Entropy

Another commonly used measure of inpurity is the information Entropy, which is formulated as

$$E(S) = \sum_{j=1}^J -p_jlog_2P_j$$

![Entropy Calculation](https://miro.medium.com/max/446/0*BdgOokoatW17zEK7.png)

Both Entropy and Gini impurity obtain the largest value when half of the data are positively labeled. Hence, smaller Entropy or Gini indicates "purer" information.

![Gini vs entropy](https://quantdare.com/wp-content/uploads/2020/11/entropy_and_gini.png)



For a detailed comparision between Gini Impuriy and Entropy, see [this blog](https://quantdare.com/decision-trees-gini-vs-entropy/).

## 2.4 Weighted Average Information Impurity and Information Gain

Note that in Section 2.2 and 2.3, the impurity measures are defined on a single set of data. However, in a tree structure, a splitting involes at least 3 datasets - set before the splitting, and 2+ subsets after the splitting.

We define the **information gain** from a splitting $X$ as: E(before X) - weighted average E(after X, subset k). And a natural weight for calculating weighted average is the size (number of observations) of the subset.
$$IG(S,X) = E(S) - \frac{\sum_{k=1}^K n_kE(S_k)}{\sum_{k=1}^K n_k}$$

Apparently, splitting that results in higher information gain is a "better" splitting.

## 2.5 Algorithm - Let computers find the "best" according to our definition and measures

Steps in ID3 algorithm:

1. It begins with the original set S as the root node.
2. On each iteration of the algorithm, it iterates through every unused variable of the set S and calculates Entropy(E) and Information gain(IG) of this variable.
3. It then selects the variable which yields the largest Information gain.
4. The set S is then split by the selected variable to produce a subset of the data.
5. The algorithm continues to recur on each subset, considering only variables never selected before, utill it reaches the preset depth of the tree.

## 2.6 Refined measures

Every measure has pros and cons, and the choice of measure can give us different results. 

### Gini Impurity
For example, An algorithm called **CART** (Classification and Regression Tree) uses Gini impurity instead of entropy to create split points. Gini impurity is easier to compute, and there is no agreement on whether Information Gain is better than Gini Index measure. See [Raileanu and  Stoffel](https://www.unine.ch/files/live/sites/imi/files/shared/documents/papers/Gini_index_fulltext.pdf) for a detailed discussion between Gini Index and Information Gain.

### Reduction in Variance
**Reduction in Variance** is a another measure when the target variable is **continuous**, i.e., regression problems. It is so-called because it uses variance as a measure for impurity.
$$\text{Population Variance}(S) = \frac{\sum_{i=1}^n (x_i-\mu)^2}{n}$$

### Chi-Square

In place of Information Gain, there are other measures of splitting quality. For example, The **CHAID** (Chi-square Automatic Interaction Detector) algorithm utilizes the **Chi-square** measure to find the best splitting.
$$\chi^2(S,X) = \sum_{k=1}^K \sum_{j=1}^J \frac{\text{observed number of label j in subset k - expected number of label j in subset k}}{\text{expected number of label j in subset k}}$$
where the $\text{expected number of label j in subset k} = n_k * \frac{\text{number of label j in set S}}{n_S}$

Under the hood, **Chi-square** is a statistic that measures whether two proportions are significantly different. Since proportion is defined for categorical variables, Chi-Square measure can not be applied directly to a continuous target.

### Gain Ratio
Information gain is biased towards choosing variables with a large number of categories.

**C4.5**, an improved algorithm of ID3, uses **Gain ratio** to reduce this bias. Gain ratio overcomes the problem with information gain by taking into account the number of branches that would result before making the split. 

$$\text{Gain Ratio} = \frac{IG}{\sum_{i=1}^K w_ilog_2w_i}$$
Where $K$ is the number of branches of a splitting and $w_i=\frac{n_i}{\sum n_i}$